In [10]:
from keras.models import load_model
from gensim.models import KeyedVectors
import numpy as np
import re
from emot.emo_unicode import UNICODE_EMOJI, EMOTICONS_EMO
from underthesea import word_tokenize



In [11]:
# Load mô hình CNN đã huấn luyện
model_sentiment = load_model("models.h5")
print("Sentiment model loaded successfully!")


Sentiment model loaded successfully!


In [12]:
file_path = 'baomoi.model.bin'
try:
    model_embedding = KeyedVectors.load_word2vec_format(file_path, binary=True)
    print("Embedding model loaded successfully!")
except Exception as e:
    print(f"Error loading embedding model: {e}")


Embedding model loaded successfully!


In [13]:
word_labels = list(model_embedding.key_to_index.keys())
max_seq = 100  
embedding_size = 400

def comment_embedding(comment):
    matrix = np.zeros((max_seq, embedding_size))
    words = comment.split()
    for i in range(min(len(words), max_seq)):
        word = words[i]
        if word in word_labels:
            matrix[i] = model_embedding[word]
    return matrix


In [14]:

def remove_emoji(text):
    for emoji in UNICODE_EMOJI.values():
        text = text.replace(emoji, "")
    for emoticon in EMOTICONS_EMO.values():
        text = text.replace(emoticon, "")
    return text

def clean_text(text):
    text = text.lower()
    text = remove_emoji(text) 
    text = re.sub(r'\d+', ' ', text)  
    text = re.sub(r'[^\w\s]', ' ', text)  
    text = re.sub(r'\s+', ' ', text).strip()  
    
    words = word_tokenize(text, format="text")
    return words

def preprocess_data(text):
    text = clean_text(text)  
    return text


0: Negative, 1: Neutral, 2: Positive

In [ ]:
text = "Hihi haha"
text_cleaned = preprocess_data(text)

matrix_embedding = np.expand_dims(comment_embedding(text_cleaned), axis=0)
matrix_embedding = np.expand_dims(matrix_embedding, axis=3)

result = model_sentiment.predict(matrix_embedding)
result_label = np.argmax(result)  

print("Label predict:", result_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Label predict: 0
